In [ ]:
import torch
import detectron2.utils.comm as comm
import json

from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.data.datasets import register_coco_instances
from detectron2.data.datasets import load_coco_json
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer
from detectron2.engine import DefaultPredictor

In [ ]:
register_coco_instances("taco_dataset_train", {}, 
                        "data/annotations_0_train.json", 
                        "/home/yaroslav/Документы/trash_segmentation_synth/data/")

register_coco_instances("taco_dataset_test", {}, 
                        "annotations_0_test.json", 
                        "/home/yaroslav/Документы/trash_segmentation_synth/data/")

In [ ]:
# load_coco_json("data/annotations_0_train.json", 
#                                   "home/yaroslav/Документы/trash_segmentation_synth/data/", 
#                                   "taco_dataset_train")

# load_coco_json("data/annotations_0_test.json", 
#                                   "home/yaroslav/Документы/trash_segmentation_synth/data/", 
#                                   "taco_dataset_test")

In [ ]:
# Создание конфигурации для модели
cfg = get_cfg()
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/retinanet_R_50_FPN_1x.yaml")
cfg.DATASETS.TRAIN = ("taco_dataset_train",)
cfg.DATASETS.TEST = ("taco_dataset_test",)
cfg.DATALOADER.NUM_WORKERS = 8
cfg.SOLVER.IMS_PER_BATCH = 8
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 1000
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
cfg.OUTPUT_DIR = "output/"

In [ ]:
def getSupercategory(category_id, coco_annotation_file_path):
    f = open(coco_annotation_file_path)
    cat_json_file = json.load(f)

    for category in cat_json_file['categories']:
        if category_id == category['id']:
            f = open('data/supercategories_2_classes.json')
            supcat_json_file = json.load(f)
            return supcat_json_file[category['supercategory']]
z = 0
for sample in DatasetCatalog.get('taco_dataset_train'):
    print(z, end="\r")
    z += 1
    for ann in sample['annotations']:
        ann['category_id'] = getSupercategory(ann['category_id'], 'data/annotations_0_train.json')

In [ ]:
DatasetCatalog.get('taco_dataset_train')

In [ ]:
for sample in DatasetCatalog.get('taco_dataset_test'):
    for ann in sample['annotations']:
        ann['category_id'] = getSupercategory(ann['category_id'], 'data/annotations_0_test.json')

In [ ]:
# Создание трейнера и обучение модели
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()
# Загрузка обученной модели
model = DefaultPredictor(cfg).model
# Сохранение обученной модели
model.save("output/model_weights.pth")